# Data cleaning

In [1]:
import pandas as pd

In [3]:
df_dogs_raw = pd.read_csv("../data/raw/20200306_hundehalter.csv")

In [4]:
df_dogs_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7841 entries, 0 to 7840
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   HALTER_ID         7841 non-null   int64  
 1   ALTER             7841 non-null   object 
 2   GESCHLECHT        7841 non-null   object 
 3   STADTKREIS        7841 non-null   int64  
 4   STADTQUARTIER     7841 non-null   int64  
 5   RASSE1            7841 non-null   object 
 6   RASSE1_MISCHLING  560 non-null    object 
 7   RASSE2            606 non-null    object 
 8   RASSE2_MISCHLING  0 non-null      float64
 9   RASSENTYP         7840 non-null   object 
 10  GEBURTSJAHR_HUND  7841 non-null   int64  
 11  GESCHLECHT_HUND   7841 non-null   object 
 12  HUNDEFARBE        7841 non-null   object 
dtypes: float64(1), int64(4), object(8)
memory usage: 796.5+ KB


In [5]:
df_dogs_raw.describe()

,HALTER_ID,STADTKREIS,STADTQUARTIER,RASSE2_MISCHLING,GEBURTSJAHR_HUND
count,7841.000000,7841.000000,7841.000000,0.0,7841.000000
mean,118143.509119,7.414998,76.727841,NaN,2012.417549
std,25677.406136,3.280150,33.239609,NaN,32.188030
min,574.000000,1.000000,8.000000,NaN,11.000000
25%,95904.000000,4.000000,44.000000,NaN,2010.000000
50%,123218.000000,8.000000,81.000000,NaN,2013.000000
75%,140615.000000,10.000000,102.000000,NaN,2016.000000
max,152331.000000,12.000000,123.000000,NaN,2020.000000


Some dog birthyears are too early, and some are small numbers (I cannot make sure if these are their age or 2000 something years). I drop these from the dataset.

In [6]:
df_dogs_raw = df_dogs_raw[df_dogs_raw["GEBURTSJAHR_HUND"] > 1995]

I calculate and age column (the dataset is from 2020)

In [7]:
df_dogs_raw["dog_age"] = 2020 - df_dogs_raw["GEBURTSJAHR_HUND"]

I drop the columns that I will not use

In [8]:
df_dogs_raw = df_dogs_raw.drop(["STADTQUARTIER", "RASSE1_MISCHLING", "RASSE2", "RASSE2_MISCHLING", "RASSENTYP"], axis=1)

In [9]:
df_dogs_raw.head()

,HALTER_ID,ALTER,GESCHLECHT,STADTKREIS,RASSE1,GEBURTSJAHR_HUND,GESCHLECHT_HUND,HUNDEFARBE,dog_age
0,574,61-70,w,2,Mischling gross,2013,w,schwarz,7
1,695,41-50,m,6,Labrador Retriever,2012,w,braun,8
2,893,71-80,w,7,Mittelschnauzer,2010,w,schwarz,10
3,916,41-50,m,3,Mischling klein,2015,w,hellbraun,5
4,1177,51-60,m,10,Shih Tzu,2011,m,schwarz/weiss,9


I give columns easier English names

In [10]:
df_dogs_raw = df_dogs_raw.rename(columns = {
    "HALTER_ID" : "owner_id", 
    "ALTER" : "owner_age_group", 
    "GESCHLECHT" : "owner_gender", 
    "STADTKREIS" : "district", 
    "RASSE1" : "dog_breed", 
    "GEBURTSJAHR_HUND" : "dog_birthyear", 
    "GESCHLECHT_HUND" : "dog_sex", 
    "HUNDEFARBE" : "dog_colour"
})

In [11]:
df_dogs_raw.head()

,owner_id,owner_age_group,owner_gender,district,dog_breed,dog_birthyear,dog_sex,dog_colour,dog_age
0,574,61-70,w,2,Mischling gross,2013,w,schwarz,7
1,695,41-50,m,6,Labrador Retriever,2012,w,braun,8
2,893,71-80,w,7,Mittelschnauzer,2010,w,schwarz,10
3,916,41-50,m,3,Mischling klein,2015,w,hellbraun,5
4,1177,51-60,m,10,Shih Tzu,2011,m,schwarz/weiss,9


I map new values to gender, breed, and colour columns. Since there are too many values, I focus on the most popular ones, and keep the German names for the rest.

In [23]:
breed_popular = df_dogs_raw.value_counts("dog_breed")[:20]
breed_popular

dog_breed
Mischling klein           659
Chihuahua                 573
Labrador Retriever        426
Mischling gross           379
Französische Bulldogge    312
Yorkshire Terrier         299
Jack Russel Terrier       276
Malteser                  223
Mops                      176
Golden Retriever          162
Dachshund                 158
Zwergspitz                148
Bolonka Zwetna            132
Border Collie             123
Beagle                    108
Pudel                      87
Pinscher                   87
Lagotto Romagnolo          86
Labrador                   83
Zwergpudel                 83
dtype: int64

In [26]:
colour_popular = df_dogs_raw.value_counts("dog_colour")[:20]
for i in colour_popular.index:
    print(i)

schwarz
tricolor
weiss
braun
schwarz/weiss
beige
schwarz/braun
braun/weiss
weiss/braun
black/tan
weiss/schwarz
rot
hellbraun
braun/schwarz
creme
golden
grau
gestromt
blondfarben
rot/weiss


Mapping dictionaires

In [28]:
sex_map = {
    "w" : "Female",
    "m" : "Male"
    }

breed_map = {
    "Mischling klein" : "Small mixed breed", 
    "Mischling gross" : "Big mixed breed", 
    "Französische Bulldogge" : "French bulldog", 
    "Malteser" : "Maltese", 
    "Mops" : "Pug", 
    "Zwergspitz" : "Pomeranian", 
    "Pudel" : "Standard poodle", 
    "Zwergpudel" : "Miniature poodle"
}

colour_map = {
    "schwarz" : "black", 
    "weiss" : "white", 
    "braun" : "brown", 
    "schwarz/weiss" : "black & white", 
    "schwarz/braun" : "black & brown", 
    "braun/weiss" : "brown & white", 
    "weiss/braun" : "white & brown",
    "black/tan" : "black & light brown", 
    "weiss/schwarz" : "white & black", 
    "rot" : "red", 
    "hellbraun" : "light brown", 
    "braun/schwarz" : "brown & black", 
    "creme" : "cream", 
    "grau" : "grey", 
    "gestromt" : "brindle", 
    "blondfarben" : "blond", 
    "rot/weiss" : "red & white"
} 


In [30]:
df_dogs_raw = df_dogs_raw.replace({
    "owner_gender" : sex_map, 
    "dog_sex" : sex_map, 
    "dog_breed" : breed_map, 
    "dog_colour" : colour_map
})
df_dogs_raw.head()

,owner_id,owner_age_group,owner_gender,district,dog_breed,dog_birthyear,dog_sex,dog_colour,dog_age
0,574,61-70,Female,2,Big mixed breed,2013,Female,black,7
1,695,41-50,Male,6,Labrador Retriever,2012,Female,brown,8
2,893,71-80,Female,7,Mittelschnauzer,2010,Female,black,10
3,916,41-50,Male,3,Small mixed breed,2015,Female,light brown,5
4,1177,51-60,Male,10,Shih Tzu,2011,Male,black & white,9


An additional column to label districts

In [33]:
district_map = {
    1 : "Altstadt", 
    2 : "Wollishofen, Leimbach, Enge", 
    3 : "Wiedikon", 
    4 : "Aussersihl", 
    5 : "Industriequartier", 
    6 : "Unterstrass, Oberstrass", 
    7 : "Funtern, Hottingen, Hirslanden, Witikon", 
    8 : "Riesbach", 
    9 : "Albisriesen, Altstetten", 
    10 : "Hoengg, Wipkingen", 
    11 : "Affoltern, Seebach, Oerlikon", 
    12 : "Schwamendingen"}

In [34]:
df_dogs_raw["distric_name"] = df_dogs_raw["district"].map(district_map)
df_dogs_raw.head()

,owner_id,owner_age_group,owner_gender,district,dog_breed,dog_birthyear,dog_sex,dog_colour,dog_age,distric_name
0,574,61-70,Female,2,Big mixed breed,2013,Female,black,7,"Wollishofen, Leimbach, Enge"
1,695,41-50,Male,6,Labrador Retriever,2012,Female,brown,8,"Unterstrass, Oberstrass"
2,893,71-80,Female,7,Mittelschnauzer,2010,Female,black,10,"Funtern, Hottingen, Hirslanden, Witikon"
3,916,41-50,Male,3,Small mixed breed,2015,Female,light brown,5,Wiedikon
4,1177,51-60,Male,10,Shih Tzu,2011,Male,black & white,9,"Hoengg, Wipkingen"


Write the dataframe to a csv to be used in the app

In [37]:
df_dogs_raw.to_csv("../data/processed/zurich_dogs.csv")

In [39]:
district_info = pd.DataFrame({
    "district" : [x for x in range(1, 13)], 
    "area" : [1.8, 11.07, 8.65, 2.9, 1.99, 5.1, 15.02, 4.81, 12.07, 9.09, 13.42, 5.97], 
    "pop" : [5817, 36025, 50104, 29023, 15804, 35355, 38663, 17818, 57077, 41017, 76975, 32654]
})

In [40]:
district_info

,district,area,pop
0,1,1.80,5817
1,2,11.07,36025
2,3,8.65,50104
3,4,2.90,29023
4,5,1.99,15804
5,6,5.10,35355
6,7,15.02,38663
7,8,4.81,17818
8,9,12.07,57077
9,10,9.09,41017


In [41]:
district_info.to_csv("../data/processed/district_info.csv")